# Online Retail
https://archive.ics.uci.edu/ml/datasets/Online+Retail

## 0. ライブラリのインポート

importできないライブラリが存在する場合、pip installでライブラリをインストールする。

In [1]:
import pandas as pd
import numpy as np
import pymysql
import pandas.io.sql as psql
import matplotlib.pyplot as plt
import scipy.spatial as sp
import time

## 1. データの読み込み

### AWSデータベースへ接続

In [2]:
con = pymysql.connect(
    host='datamix-school-material.csbsmnjyxb52.ap-northeast-1.rds.amazonaws.com',
    user='user1',
    password='user1',
    db='online_retail',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor)
con

### SQLによるデータ抽出
<b>各顧客（CustomerID）に対して、それぞれの商品（StockCode）を、何個（Quantity）卸したのかを集計する。</b>

- 必要なカラムはなにか。
- 除外しておいた方が良いデータはなにか。

In [3]:
sql = """
SELECT CustomerID, StockCode,
MAX(Description) AS description, 
SUM(Quantity) AS quantity 
FROM sales_log 
WHERE CustomerID <> '' 
AND description <> '' 
AND Quantity > 0  
AND Unitprice > 0 
GROUP BY CustomerID, StockCode; 
"""

In [4]:
data = psql.read_sql(con=con, sql=sql)

In [5]:
data.head()

,CustomerID,StockCode,description,quantity
0,12346,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215.0
1,12347,16008,SMALL FOLDING SCISSOR(POINTED EDGE),24.0
2,12347,17021,NAMASTE SWAGAT INCENSE,36.0
3,12347,20665,RED RETROSPOT PURSE,6.0
4,12347,20719,WOODLAND CHARLOTTE BAG,10.0


In [6]:
data.shape

(128406, 4)

## 2. 顧客ID×商品コードのテーブルを作成

### Pandasを使ったデータ加工
<b>Pandasを使って、顧客ID（CustomerID）×商品コード（StockCode）のテーブルを作成する。</b>

- Pandasのどの関数を使ったらよいか。
- テーブルにおける、行・列・行列内の値、は何を指定すればよいか。

In [7]:
customer_stock_matrix = pd.pivot_table(data, index="CustomerID", columns="description", values="quantity")
customer_stock_matrix = customer_stock_matrix.fillna(0)

In [8]:
customer_stock_matrix.head()

description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12348,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12352,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
customer_stock_matrix.shape

(2931, 3352)

## 3. レコメンデーション用の関数を作成

### レコメンドする関数を作成する

<b>ある顧客に対して、新たな商品をレコメンデーションする関数を完成させる</b>

In [10]:
def recommend(target_customer_id, customer_stock_matrix, user_top_N, recommend_top_N):
    
    # customer_stock_matrixから、index, columns, 行列を取得する
    stock_list = customer_stock_matrix.columns
    customer_list = customer_stock_matrix.index
    customer_stock_matrix_array = np.array(customer_stock_matrix)
    
    # レコメンドしたい顧客IDを引数とした際に、顧客リストから、レコメンドしたい顧客のindexを取り出す
    for index in range(len(customer_list)):
        if customer_list[index] == target_customer_id:
            target_customer_index = index
    
    # レコメンドしたい顧客と、全顧客との類似度を計算する
    user_similarity = []
    target_user_row = customer_stock_matrix_array[target_customer_index]
    
    for row in customer_stock_matrix_array:
        # cosine類似度　= 1 - cosine距離
        similarity = 1 - sp.distance.cosine(target_user_row, row)
        user_similarity.append(similarity)

    user_similarity = np.array(user_similarity)
    
    # レコメンドしたい顧客に対し、コサイン類似度の高いユーザー上位N人を対象に、商品の平均類似度を計算する
    
    ## ユーザー上位N人の類似度と購入数量行列を取得する
    idx = user_similarity.argsort()[::-1][1:user_top_N + 1]
    selected_user_similarity = user_similarity[idx]
    selected_matrix = customer_stock_matrix_array[idx]
    
    ## ユーザー上位N人を対象に、商品ごとの平均類似度を計算する
    avg_score = []
    for col_idx in range(selected_matrix.shape[1]):
        weight_score = sum(selected_matrix[:, col_idx] * selected_user_similarity)
        similarity_sum = sum(selected_user_similarity[selected_user_similarity > 0])
        avg_score.append(weight_score / similarity_sum)
    avg_score = np.array(avg_score)
    
    # レコメンドする必要がない、既に購入済の商品リストを抽出する
    already_ordered_product = stock_list[np.where(customer_stock_matrix[customer_stock_matrix.index == target_customer_id])[1]]
    
    # 平均類度の高い上位N商品をオススメとして表示する
    counter = 0
    print("■レコメンド商品リスト:")
    for recommended_product in stock_list[avg_score.argsort()[::-1]]:
        if recommended_product not in already_ordered_product:
            print(counter + 1 ,recommended_product)
            counter += 1
            if recommend_top_N <= counter:
                break

In [11]:
# 関数の実行 (ある顧客に対するレコメンド商品・上位10商品を表示する)
# 引数は対象の顧客ID, 顧客ID×商品コードのテーブル、類似度の高い上位ユーザー数、類似度の高い上位商品数
recommend('12346', customer_stock_matrix, 20, 10)

■レコメンド商品リスト:
1 GRAND CHOCOLATECANDLE
2 SMALL CERAMIC TOP STORAGE JAR 
3 LARGE CERAMIC TOP STORAGE JAR
4 COFFEE MUG CAT + BIRD DESIGN
5 JAM JAR WITH PINK LID
6 PACK OF 72 RETROSPOT CAKE CASES
7 JAM MAKING SET PRINTED
8 GREEN REGENCY TEACUP AND SAUCER
9 ROSES REGENCY TEACUP AND SAUCER 
10 REGENCY TEA PLATE GREEN 
